# Código para concatenar todas as Bases de Dados a serem utilizadas na Previsão da Inflação de Bens Industriais numa Base de Dados única

As bases de dados de todos os indicadores a serem utilizadas na previsão da inflação de bens industriais foram analisadas e tratadas em diferentes códigos. A partir destes códigos foram gerados arquivos em formato csv que serão os inputs para este código.

Indicadores a serem utilizados:

IPA

PIM-PF

ICI

NUCI

Nível de Estoques

IBC-Br

IC-Br

Consumo Aparente

Consumo de Energia Elétrica - Indústria

In [48]:
# Importando bibliotecas necessárias:

import requests # biblioteca para acessar e interagir com APIs e páginas da web
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from seaborn import set_theme

In [49]:
#Bibliotecas de tratamento de estatistico de séries e de manipulação de datas e horários:

from statsmodels.tsa.seasonal import seasonal_decompose

from dateutil.parser import parse

In [50]:
#Biblioteca para Análise de Estacionariedade por Teste de ADF

from statsmodels.tsa.stattools import adfuller

In [51]:
# Biblioteca para Geração de Gráfico de Função de Autocorrelação e Autocorrelação Parcial

from statsmodels.graphics.tsaplots import plot_acf

from statsmodels.graphics.tsaplots import plot_pacf

In [52]:
# Biblioteca para Conversão de Formato do Texto do Nome das Colunas para a Forma Normalizada de Decomposição:

import unicodedata

### Função para remoção de acentos e cedilhas dos nomes das colunas para evitar problemas futuros no uso de redes neurais artificiais

In [53]:
# Função para remoção de acentos e cedilha:

def remover_acentos_e_cedilha(texto):
    # Primeiro, normaliza para a forma NFD (Canonical Decomposition Form)
    texto_normalizado = unicodedata.normalize('NFD', texto)

    # Codifica para ASCII e decodifica para remover os diacríticos
    texto_sem_acentos = texto_normalizado.encode('ascii', 'ignore').decode('utf-8')

    # Troca 'ç' por 'c' e 'Ç' por 'C'
    texto_final = texto_sem_acentos.replace('ç', 'c').replace('Ç', 'C')

    return texto_final

## Carregamento de Dados

### IPA Desagregado - Proxy do Custo de Produção da Indústria de Transformação

Este IPA foi calculado utilizando os coeficientes técnicos informados na Matriz de Insumo Produto do IBGE (2015). Para mais detalhes, ver arquivo em Python "Code_GetData_IPA".

In [54]:
# Carregamento de Dados a partir de um arquivo CSV:

df_ipa_desag = pd.read_csv('base_dados_ipa_desag.csv')
df_ipa_desag.head()

,Mes,Agricultura e Pecuaria,Carvao Mineral,Petroleo e Gas Natural,Minerais Metalicos,Minerais Nao-Metalicos,Produtos Alimenticios,Bebidas,Fumo Processado e Produtos do Fumo,Produtos Texteis,...,Metalurgia Basica,Produtos de Metal,"Equipamentos de Informatica, Produtos Eletronicos e Opticos","Maquinas, Aparelhos e Materiais Eletricos",Maquinas e Equipamentos,"Veiculos Automotores, Reboques, Carrocerias e Autopecas","Embarcacoes, Veiculos Ferroviarios e Outros Equipamentos de Transporte",Moveis,IPCA Serv Var Mensal (%),Proxy - Custo Prod Ind Transf Var Mensal%
0,2005-01-31,-0.63,-0.18,NaN,-0.36,1.10,-0.38,1.86,3.15,1.14,...,1.63,1.14,-6.24,1.21,1.45,2.02,0.09,0.17,0.46,0.250514
1,2005-02-28,1.29,0.00,NaN,-4.20,-3.72,0.51,-0.31,0.02,-0.38,...,0.89,-0.43,-0.99,1.34,0.32,0.77,0.84,1.44,1.82,0.409522
2,2005-03-31,3.59,0.00,NaN,2.00,3.31,1.95,0.74,4.62,-0.14,...,0.10,0.66,1.83,0.46,0.16,-0.21,0.56,1.14,0.51,0.754040
3,2005-04-30,-1.60,0.00,NaN,15.87,0.69,-0.74,0.83,0.06,-0.34,...,0.09,0.84,1.48,1.54,0.34,1.12,-0.14,1.59,0.35,0.790196
4,2005-05-31,-2.76,3.22,NaN,2.30,-3.04,-0.26,0.77,-0.14,-0.48,...,-1.09,0.16,-4.96,0.47,1.72,0.43,-3.54,0.09,0.28,-0.343708


In [55]:
# Simplificando o dataframe para somente as colunas relevantes para o Proxy de Custo de Produção Industrial:

df_proxy_custo_ipa = df_ipa_desag[["Mes", "Proxy - Custo Prod Ind Transf Var Mensal%"]]
df_proxy_custo_ipa.head()

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%
0,2005-01-31,0.250514
1,2005-02-28,0.409522
2,2005-03-31,0.754040
3,2005-04-30,0.790196
4,2005-05-31,-0.343708


In [56]:
df_proxy_custo_ipa

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%
0,2005-01-31,0.250514
1,2005-02-28,0.409522
2,2005-03-31,0.754040
3,2005-04-30,0.790196
4,2005-05-31,-0.343708
...,...,...
235,2024-08-31,0.607579
236,2024-09-30,0.775323
237,2024-10-31,0.638631
238,2024-11-30,0.857047


In [57]:
# Criação de um dataframe onde serão concatenados os dados de todos os indicadores:

df_base = df_proxy_custo_ipa.copy()

df_base.head()

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%
0,2005-01-31,0.250514
1,2005-02-28,0.409522
2,2005-03-31,0.754040
3,2005-04-30,0.790196
4,2005-05-31,-0.343708


### PIM-PF (Sem Ajuste Sazonal)

In [58]:
# Carregamento de Dados a partir de outro arquivo CSV:

df_pimpf_bruto = pd.read_csv('base_dados_pimpf.csv')
df_pimpf_bruto.head()

,Mês,U. Medida,3 Indústrias de transformação,3.10 Fabricação de produtos alimentícios,3.11 Fabricação de bebidas,3.12 Fabricação de produtos do fumo,3.13 Fabricação de produtos têxteis,3.14 Confecção de artigos do vestuário e acessórios,"3.15 Preparação de couros e fabricação de artefatos de couro, artigos para viagem e calçados",3.16 Fabricação de produtos de madeira,...,"3.25 Fabricação de produtos de metal, exceto máquinas e equipamentos","3.26 Fabricação de equipamentos de informática, produtos eletrônicos e ópticos","3.27 Fabricação de máquinas, aparelhos e materiais elétricos",3.28 Fabricação de máquinas e equipamentos,"3.29 Fabricação de veículos automotores, reboques e carrocerias","3.30 Fabricação de outros equipamentos de transporte, exceto veículos automotores",3.31 Fabricação de móveis,3.32 Fabricação de produtos diversos,"3.33 Manutenção, reparação e instalação de máquinas e equipamentos",Ind Transf Sem Ajuste Sazonal Var Men (%)
0,2002-02-28,Número-índice,82.08498,76.09358,57.82431,109.88529,170.89122,157.81697,160.73440,97.90458,...,115.18651,99.65788,88.55418,66.73521,74.06187,56.04452,114.46782,90.41531,NaN,-3.597425
1,2002-03-31,Número-índice,90.96360,80.99840,60.67149,206.75265,184.34960,185.05559,191.27271,111.51380,...,118.69180,116.03333,94.94183,73.86466,85.37216,66.61859,134.97946,95.92660,NaN,10.816376
2,2002-04-30,Número-índice,93.77772,86.08528,62.63140,236.35534,192.55466,197.93528,205.26340,114.97550,...,131.88761,127.42721,103.15985,76.45942,91.68735,74.70428,145.59466,98.29857,NaN,3.093677
3,2002-05-31,Número-índice,94.70631,94.72804,62.29452,220.23175,183.06361,190.78067,197.63409,115.73291,...,124.28954,117.11845,94.67316,77.85233,85.92352,72.37397,136.07957,97.50967,NaN,0.990203
4,2002-06-30,Número-índice,91.07112,99.62395,60.47399,187.03269,173.72159,178.64770,153.42793,116.55992,...,113.93432,106.02189,89.45175,71.52699,77.98911,66.69482,124.92651,90.82665,NaN,-3.838382


In [59]:
# Remoção de acentos e cedilha dos nomes das colunas:

# Aplicação da função para renomear as colunas para letras sem acento e cedilha
# List comprehension para aplicar a função a cada nome de coluna

df_pimpf_bruto.columns = [remover_acentos_e_cedilha(col) for col in df_pimpf_bruto.columns]

print("\nDataFrame com Nomes de Colunas Limpos:")
print(df_pimpf_bruto.head())
print("\nNomes das colunas limpos:", df_pimpf_bruto.columns.tolist())


DataFrame com Nomes de Colunas Limpos:
          Mes      U. Medida  3 Industrias de transformacao  \
0  2002-02-28  Número-índice                       82.08498   
1  2002-03-31  Número-índice                       90.96360   
2  2002-04-30  Número-índice                       93.77772   
3  2002-05-31  Número-índice                       94.70631   
4  2002-06-30  Número-índice                       91.07112   

   3.10 Fabricacao de produtos alimenticios  3.11 Fabricacao de bebidas  \
0                                  76.09358                    57.82431   
1                                  80.99840                    60.67149   
2                                  86.08528                    62.63140   
3                                  94.72804                    62.29452   
4                                  99.62395                    60.47399   

   3.12 Fabricacao de produtos do fumo  3.13 Fabricacao de produtos texteis  \
0                            109.88529             

In [60]:
#Criando um novo Dataframe com o mesmo período de tempo do DataFrame df_proxy_custo_ipa (janeiro/2005 - dezembro/2024):

df_pimpf = df_pimpf_bruto.iloc[35:275,:].copy()

# Reset do índice do DataFrame df_pimpf:
df_pimpf.reset_index(drop=True, inplace=True)

df_pimpf.head()

,Mes,U. Medida,3 Industrias de transformacao,3.10 Fabricacao de produtos alimenticios,3.11 Fabricacao de bebidas,3.12 Fabricacao de produtos do fumo,3.13 Fabricacao de produtos texteis,3.14 Confeccao de artigos do vestuario e acessorios,"3.15 Preparacao de couros e fabricacao de artefatos de couro, artigos para viagem e calcados",3.16 Fabricacao de produtos de madeira,...,"3.25 Fabricacao de produtos de metal, exceto maquinas e equipamentos","3.26 Fabricacao de equipamentos de informatica, produtos eletronicos e opticos","3.27 Fabricacao de maquinas, aparelhos e materiais eletricos",3.28 Fabricacao de maquinas e equipamentos,"3.29 Fabricacao de veiculos automotores, reboques e carrocerias","3.30 Fabricacao de outros equipamentos de transporte, exceto veiculos automotores",3.31 Fabricacao de moveis,3.32 Fabricacao de produtos diversos,"3.33 Manutencao, reparacao e instalacao de maquinas e equipamentos",Ind Transf Sem Ajuste Sazonal Var Men (%)
0,2005-01-31,Número-índice,95.05259,87.33044,72.31495,61.56038,171.35507,127.70445,167.93439,128.21706,...,114.67109,114.84833,103.72243,82.68131,101.84683,77.34019,112.64878,114.26704,NaN,-4.469845
1,2005-02-28,Número-índice,89.74077,78.90929,62.83586,72.56080,177.75170,138.80234,157.10322,121.81360,...,118.83726,111.49130,103.45042,85.46727,103.95275,74.15256,108.24275,102.67646,NaN,-5.588296
2,2005-03-31,Número-índice,103.53528,88.16565,69.00404,170.49287,189.45694,171.46116,177.84386,141.80360,...,135.30579,145.68401,122.41010,97.20085,122.67677,97.67833,135.17333,124.01239,NaN,15.371508
3,2005-04-30,Número-índice,101.99578,89.83269,71.91775,234.12166,192.77200,171.37760,169.53971,135.71321,...,125.45827,154.43533,116.28692,90.73390,117.42237,91.37191,132.40813,109.50158,NaN,-1.486933
4,2005-05-31,Número-índice,106.75110,98.84297,65.99083,266.38561,196.08314,171.73565,166.13469,136.13965,...,126.06281,174.52414,120.82201,91.75651,119.35004,95.10782,145.17488,104.16205,NaN,4.662271


In [61]:
df_pimpf

,Mes,U. Medida,3 Industrias de transformacao,3.10 Fabricacao de produtos alimenticios,3.11 Fabricacao de bebidas,3.12 Fabricacao de produtos do fumo,3.13 Fabricacao de produtos texteis,3.14 Confeccao de artigos do vestuario e acessorios,"3.15 Preparacao de couros e fabricacao de artefatos de couro, artigos para viagem e calcados",3.16 Fabricacao de produtos de madeira,...,"3.25 Fabricacao de produtos de metal, exceto maquinas e equipamentos","3.26 Fabricacao de equipamentos de informatica, produtos eletronicos e opticos","3.27 Fabricacao de maquinas, aparelhos e materiais eletricos",3.28 Fabricacao de maquinas e equipamentos,"3.29 Fabricacao de veiculos automotores, reboques e carrocerias","3.30 Fabricacao de outros equipamentos de transporte, exceto veiculos automotores",3.31 Fabricacao de moveis,3.32 Fabricacao de produtos diversos,"3.33 Manutencao, reparacao e instalacao de maquinas e equipamentos",Ind Transf Sem Ajuste Sazonal Var Men (%)
0,2005-01-31,Número-índice,95.05259,87.33044,72.31495,61.56038,171.35507,127.70445,167.93439,128.21706,...,114.67109,114.84833,103.72243,82.68131,101.84683,77.34019,112.64878,114.26704,NaN,-4.469845
1,2005-02-28,Número-índice,89.74077,78.90929,62.83586,72.56080,177.75170,138.80234,157.10322,121.81360,...,118.83726,111.49130,103.45042,85.46727,103.95275,74.15256,108.24275,102.67646,NaN,-5.588296
2,2005-03-31,Número-índice,103.53528,88.16565,69.00404,170.49287,189.45694,171.46116,177.84386,141.80360,...,135.30579,145.68401,122.41010,97.20085,122.67677,97.67833,135.17333,124.01239,NaN,15.371508
3,2005-04-30,Número-índice,101.99578,89.83269,71.91775,234.12166,192.77200,171.37760,169.53971,135.71321,...,125.45827,154.43533,116.28692,90.73390,117.42237,91.37191,132.40813,109.50158,NaN,-1.486933
4,2005-05-31,Número-índice,106.75110,98.84297,65.99083,266.38561,196.08314,171.73565,166.13469,136.13965,...,126.06281,174.52414,120.82201,91.75651,119.35004,95.10782,145.17488,104.16205,NaN,4.662271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2024-08-31,Número-índice,113.36023,118.59058,99.21616,108.39551,115.77377,107.08378,117.74449,110.97343,...,110.45477,110.57598,105.53657,105.91661,118.02828,141.19725,119.90163,99.87763,105.23736,1.384228
236,2024-09-30,Número-índice,109.57254,112.79471,102.50581,86.10032,111.28624,104.12312,110.10783,102.65992,...,104.66069,105.18280,105.18248,99.07009,115.60123,119.51898,116.14794,96.31010,98.42536,-3.341286
237,2024-10-31,Número-índice,113.99970,114.49032,107.06575,78.42083,119.02343,121.40757,122.97063,108.34483,...,110.87586,107.06042,115.29679,106.06395,125.00663,123.06594,127.62731,105.26246,99.20654,4.040392
238,2024-11-30,Número-índice,103.95266,102.26901,104.86737,64.57226,111.25488,112.84687,110.25497,103.20829,...,104.58675,100.73412,105.22689,101.03314,108.98648,118.08561,121.14067,97.28565,96.88729,-8.813216


In [62]:
# Concatenando a coluna "Ind Transf Sem Ajuste Sazonal Var Men (%)" do df_pimpf ao dataframe df_base

df_base = df_base.merge(
    df_pimpf[["Mes", "Ind Transf Sem Ajuste Sazonal Var Men (%)"]],
    on="Mes",
    how="left"
)

df_base.head()

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%,Ind Transf Sem Ajuste Sazonal Var Men (%)
0,2005-01-31,0.250514,-4.469845
1,2005-02-28,0.409522,-5.588296
2,2005-03-31,0.754040,15.371508
3,2005-04-30,0.790196,-1.486933
4,2005-05-31,-0.343708,4.662271


### ICI (Índice de Confiança da Indústria), NUCI (Nível de Utilização da Capacidade Instalada) e Nível de Estoques da Indústria de Transformação

In [63]:
# Carregamento de Dados a partir de outro arquivo CSV:

df_ici_ind = pd.read_csv('base_dados_ici_ind.csv')
df_ici_ind

,Mês,ICI,ISA,IE
0,2001-01-31,105.7,107.3,103.9
1,2001-02-28,107.0,103.7,110.1
2,2001-03-31,103.4,100.4,106.3
3,2001-04-30,101.7,97.4,106.0
4,2001-05-31,98.3,95.1,101.6
...,...,...,...,...
283,2024-08-31,101.5,103.4,99.7
284,2024-09-30,100.4,102.9,97.9
285,2024-10-31,99.9,102.7,97.1
286,2024-11-30,99.1,101.8,96.3


In [64]:
# Remoção de acentos e cedilha dos nomes das colunas:

# Aplicação da função para renomear as colunas para letras sem acento e cedilha
# List comprehension para aplicar a função a cada nome de coluna

df_ici_ind.columns = [remover_acentos_e_cedilha(col) for col in df_ici_ind.columns]

print("\nDataFrame com Nomes de Colunas Limpos:")
print(df_ici_ind.head())
print("\nNomes das colunas limpos:", df_ici_ind.columns.tolist())


DataFrame com Nomes de Colunas Limpos:
          Mes    ICI    ISA     IE
0  2001-01-31  105.7  107.3  103.9
1  2001-02-28  107.0  103.7  110.1
2  2001-03-31  103.4  100.4  106.3
3  2001-04-30  101.7   97.4  106.0
4  2001-05-31   98.3   95.1  101.6

Nomes das colunas limpos: ['Mes', 'ICI', 'ISA', 'IE']


In [65]:
df_ici_ind

,Mes,ICI,ISA,IE
0,2001-01-31,105.7,107.3,103.9
1,2001-02-28,107.0,103.7,110.1
2,2001-03-31,103.4,100.4,106.3
3,2001-04-30,101.7,97.4,106.0
4,2001-05-31,98.3,95.1,101.6
...,...,...,...,...
283,2024-08-31,101.5,103.4,99.7
284,2024-09-30,100.4,102.9,97.9
285,2024-10-31,99.9,102.7,97.1
286,2024-11-30,99.1,101.8,96.3


In [66]:
df_ici_ind.iloc[48:,:]

,Mes,ICI,ISA,IE
48,2005-01-31,106.8,106.8,106.8
49,2005-02-28,106.6,103.4,109.6
50,2005-03-31,103.5,100.2,106.6
51,2005-04-30,102.1,97.7,106.3
52,2005-05-31,99.8,96.2,103.3
...,...,...,...,...
283,2024-08-31,101.5,103.4,99.7
284,2024-09-30,100.4,102.9,97.9
285,2024-10-31,99.9,102.7,97.1
286,2024-11-30,99.1,101.8,96.3


In [67]:
#Criando um novo Dataframe com o mesmo período de tempo do DataFrame df_proxy_custo_ipa (janeiro/2005 - dezembro/2024):

df_ici_ind05 = df_ici_ind.iloc[48:,:].copy()

# Reset do índice do DataFrame df_pimpf:
df_ici_ind05.reset_index(drop=True, inplace=True)

df_ici_ind05

,Mes,ICI,ISA,IE
0,2005-01-31,106.8,106.8,106.8
1,2005-02-28,106.6,103.4,109.6
2,2005-03-31,103.5,100.2,106.6
3,2005-04-30,102.1,97.7,106.3
4,2005-05-31,99.8,96.2,103.3
...,...,...,...,...
235,2024-08-31,101.5,103.4,99.7
236,2024-09-30,100.4,102.9,97.9
237,2024-10-31,99.9,102.7,97.1
238,2024-11-30,99.1,101.8,96.3


In [68]:
# Concatenando a coluna "ICI" do df_pimpf ao dataframe df_base

df_base = df_base.merge(
    df_ici_ind05[["Mes", "ICI"]],
    on="Mes",
    how="left"
)

df_base.head()

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%,Ind Transf Sem Ajuste Sazonal Var Men (%),ICI
0,2005-01-31,0.250514,-4.469845,106.8
1,2005-02-28,0.409522,-5.588296,106.6
2,2005-03-31,0.754040,15.371508,103.5
3,2005-04-30,0.790196,-1.486933,102.1
4,2005-05-31,-0.343708,4.662271,99.8


In [69]:
# Carregamento de Dados a partir de outro arquivo CSV:

df_nuci_est_bruto = pd.read_csv('base_dados_ici_bruto.csv')

df_nuci_est_bruto

,Mês,Demanda Total,Demanda interna,Demanda externa,Nível de estoques,Situação atual dos negócios,NUCI,Demanda Total Prevista,Demanda interna prevista,Demanda externa prevista,Produção prevista,Emprego previsto,Tendência dos negócios
0,2001-01-31,104.0,103.8,97.7,96.8,117.9,82.9,125.8,128.2,124.3,122.1,107.9,149.3
1,2001-02-28,100.0,100.6,97.3,93.7,116.6,82.7,136.7,142.6,126.1,138.5,112.5,150.0
2,2001-03-31,96.0,96.6,96.4,91.5,113.5,82.6,127.3,131.9,119.8,129.2,108.7,149.9
3,2001-04-30,94.1,94.1,97.3,89.0,110.3,81.9,131.4,133.2,121.2,130.2,107.4,148.8
4,2001-05-31,91.2,90.1,93.8,88.3,104.9,80.8,120.2,122.8,112.9,120.6,105.4,142.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2024-08-31,99.0,100.6,94.8,96.2,107.0,83.2,118.2,120.2,108.2,115.7,109.7,133.1
284,2024-09-30,97.1,96.4,98.6,96.0,107.2,83.4,114.1,116.3,106.2,116.5,106.5,126.6
285,2024-10-31,100.5,100.4,96.7,94.5,108.0,82.5,115.6,118.4,103.3,111.9,107.8,127.8
286,2024-11-30,97.4,97.6,94.1,95.5,103.3,81.7,113.7,115.4,103.3,111.5,107.1,124.9


In [70]:
# Remoção de acentos e cedilha dos nomes das colunas:

# Aplicação da função para renomear as colunas para letras sem acento e cedilha
# List comprehension para aplicar a função a cada nome de coluna

df_nuci_est_bruto.columns = [remover_acentos_e_cedilha(col) for col in df_nuci_est_bruto.columns]

print("\nDataFrame com Nomes de Colunas Limpos:")
print(df_nuci_est_bruto.head())
print("\nNomes das colunas limpos:", df_nuci_est_bruto.columns.tolist())


DataFrame com Nomes de Colunas Limpos:
          Mes  Demanda Total  Demanda interna  Demanda externa  \
0  2001-01-31          104.0            103.8             97.7   
1  2001-02-28          100.0            100.6             97.3   
2  2001-03-31           96.0             96.6             96.4   
3  2001-04-30           94.1             94.1             97.3   
4  2001-05-31           91.2             90.1             93.8   

   Nivel de estoques  Situacao atual dos negocios  NUCI  \
0               96.8                        117.9  82.9   
1               93.7                        116.6  82.7   
2               91.5                        113.5  82.6   
3               89.0                        110.3  81.9   
4               88.3                        104.9  80.8   

   Demanda Total Prevista  Demanda interna prevista  Demanda externa prevista  \
0                   125.8                     128.2                     124.3   
1                   136.7                     

In [71]:
# Concatenando as colunas "Nivel de estoques" e "NUCI" do df_nuci_est_bruto ao dataframe df_base

df_base = df_base.merge(
    df_nuci_est_bruto[["Mes", "Nivel de estoques", "NUCI"]],
    on="Mes",
    how="left"
)

df_base

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%,Ind Transf Sem Ajuste Sazonal Var Men (%),ICI,Nivel de estoques,NUCI
0,2005-01-31,0.250514,-4.469845,106.8,94.8,84.3
1,2005-02-28,0.409522,-5.588296,106.6,92.8,83.9
2,2005-03-31,0.754040,15.371508,103.5,91.6,83.4
3,2005-04-30,0.790196,-1.486933,102.1,90.7,82.2
4,2005-05-31,-0.343708,4.662271,99.8,90.1,81.5
...,...,...,...,...,...,...
235,2024-08-31,0.607579,1.384228,101.5,96.2,83.2
236,2024-09-30,0.775323,-3.341286,100.4,96.0,83.4
237,2024-10-31,0.638631,4.040392,99.9,94.5,82.5
238,2024-11-30,0.857047,-8.813216,99.1,95.5,81.7


### IBC-Br

In [72]:
# Carregamento de Dados a partir de outro arquivo CSV:

df_ibc_bruto = pd.read_csv('base_dados_ibc.csv')
df_ibc_bruto

,Mês,IBC-BR - Número índice,IBC-BR dessazonal - Número Índice,IBC-BR Variação Mensal (%),IBC-BR Dessazonal Variação Mensal (%)
0,2003-02-28,98.67000,102.10000,2.620905,1.632491
1,2003-03-31,103.41000,101.71000,4.803892,-0.381978
2,2003-04-30,102.19000,101.09000,-1.179770,-0.609576
3,2003-05-31,100.30000,99.98000,-1.849496,-1.098031
4,2003-06-30,98.58000,100.15000,-1.714855,0.170034
...,...,...,...,...,...
260,2024-10-31,109.49845,107.96121,2.037204,-0.043367
261,2024-11-30,105.76017,108.00410,-3.414003,0.039727
262,2024-12-31,104.44111,107.32500,-1.247218,-0.628772
263,2025-01-31,103.05176,108.31633,-1.330271,0.923671


In [73]:
# Remoção de acentos e cedilha dos nomes das colunas:

# Aplicação da função para renomear as colunas para letras sem acento e cedilha
# List comprehension para aplicar a função a cada nome de coluna

df_ibc_bruto.columns = [remover_acentos_e_cedilha(col) for col in df_ibc_bruto.columns]

print("\nDataFrame com Nomes de Colunas Limpos:")
print(df_ibc_bruto .head())
print("\nNomes das colunas limpos:", df_ibc_bruto.columns.tolist())


DataFrame com Nomes de Colunas Limpos:
          Mes  IBC-BR - Numero indice  IBC-BR dessazonal - Numero Indice  \
0  2003-02-28                   98.67                             102.10   
1  2003-03-31                  103.41                             101.71   
2  2003-04-30                  102.19                             101.09   
3  2003-05-31                  100.30                              99.98   
4  2003-06-30                   98.58                             100.15   

   IBC-BR Variacao Mensal (%)  IBC-BR Dessazonal Variacao Mensal (%)  
0                    2.620905                               1.632491  
1                    4.803892                              -0.381978  
2                   -1.179770                              -0.609576  
3                   -1.849496                              -1.098031  
4                   -1.714855                               0.170034  

Nomes das colunas limpos: ['Mes', 'IBC-BR - Numero indice', 'IBC-BR dessazon

In [77]:
df_ibc_bruto.iloc[23:263,:]

,Mes,IBC-BR - Numero indice,IBC-BR dessazonal - Numero Indice,IBC-BR Variacao Mensal (%),IBC-BR Dessazonal Variacao Mensal (%)
23,2005-01-31,103.52000,109.57000,-3.756043,0.642969
24,2005-02-28,104.00000,110.23000,0.463679,0.602355
25,2005-03-31,115.42000,110.28000,10.980769,0.045360
26,2005-04-30,112.35000,111.26000,-2.659851,0.888647
27,2005-05-31,110.86000,111.02000,-1.326213,-0.215711
...,...,...,...,...,...
258,2024-08-31,110.24967,107.18603,-1.342374,0.279575
259,2024-09-30,107.31228,108.00805,-2.664307,0.766910
260,2024-10-31,109.49845,107.96121,2.037204,-0.043367
261,2024-11-30,105.76017,108.00410,-3.414003,0.039727


In [78]:
#Criando um novo Dataframe com o mesmo período de tempo do DataFrame df_proxy_custo_ipa (janeiro/2005 - dezembro/2024):

df_ibc = df_ibc_bruto.iloc[23:263,:].copy()

# Reset do índice do DataFrame::
df_ibc.reset_index(drop=True, inplace=True)

df_ibc

,Mes,IBC-BR - Numero indice,IBC-BR dessazonal - Numero Indice,IBC-BR Variacao Mensal (%),IBC-BR Dessazonal Variacao Mensal (%)
0,2005-01-31,103.52000,109.57000,-3.756043,0.642969
1,2005-02-28,104.00000,110.23000,0.463679,0.602355
2,2005-03-31,115.42000,110.28000,10.980769,0.045360
3,2005-04-30,112.35000,111.26000,-2.659851,0.888647
4,2005-05-31,110.86000,111.02000,-1.326213,-0.215711
...,...,...,...,...,...
235,2024-08-31,110.24967,107.18603,-1.342374,0.279575
236,2024-09-30,107.31228,108.00805,-2.664307,0.766910
237,2024-10-31,109.49845,107.96121,2.037204,-0.043367
238,2024-11-30,105.76017,108.00410,-3.414003,0.039727


In [79]:
# Concatenando a coluna "IBC-BR Variacao Mensal (%)" do df_ibc ao dataframe df_base

df_base = df_base.merge(
    df_ibc[["Mes", "IBC-BR Variacao Mensal (%)"]],
    on="Mes",
    how="left"
)

df_base

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%,Ind Transf Sem Ajuste Sazonal Var Men (%),ICI,Nivel de estoques,NUCI,IBC-BR Variacao Mensal (%)
0,2005-01-31,0.250514,-4.469845,106.8,94.8,84.3,-3.756043
1,2005-02-28,0.409522,-5.588296,106.6,92.8,83.9,0.463679
2,2005-03-31,0.754040,15.371508,103.5,91.6,83.4,10.980769
3,2005-04-30,0.790196,-1.486933,102.1,90.7,82.2,-2.659851
4,2005-05-31,-0.343708,4.662271,99.8,90.1,81.5,-1.326213
...,...,...,...,...,...,...,...
235,2024-08-31,0.607579,1.384228,101.5,96.2,83.2,-1.342374
236,2024-09-30,0.775323,-3.341286,100.4,96.0,83.4,-2.664307
237,2024-10-31,0.638631,4.040392,99.9,94.5,82.5,2.037204
238,2024-11-30,0.857047,-8.813216,99.1,95.5,81.7,-3.414003


### IC-Br (Índice de Commodities)

In [80]:
# Carregamento de Dados a partir de outro arquivo CSV:

df_ic_bruto = pd.read_csv('base_dados_ic.csv')
df_ic_bruto

,Mês,Composto R$,Agropecuária R$,Metal R$,Energia R$,IC - Composto Var Men (%)
0,1998-02-28,38.98,46.62,30.64,18.25,-0.535851
1,1998-03-31,38.16,45.40,30.70,18.11,-2.103643
2,1998-04-30,38.10,44.87,30.92,19.57,-0.157233
3,1998-05-31,38.21,45.28,30.18,19.34,0.288714
4,1998-06-30,36.67,43.31,29.57,18.48,-4.030359
...,...,...,...,...,...,...
322,2024-12-31,477.11,531.59,549.76,209.73,6.736018
323,2025-01-31,482.57,537.72,543.84,216.72,1.144390
324,2025-02-28,461.10,510.22,541.60,204.28,-4.449095
325,2025-03-31,447.04,484.87,560.79,201.16,-3.049230


In [81]:
# Remoção de acentos e cedilha dos nomes das colunas:

# Aplicação da função para renomear as colunas para letras sem acento e cedilha
# List comprehension para aplicar a função a cada nome de coluna

df_ic_bruto.columns = [remover_acentos_e_cedilha(col) for col in df_ic_bruto.columns]

print("\nDataFrame com Nomes de Colunas Limpos:")
print(df_ic_bruto .head())
print("\nNomes das colunas limpos:", df_ic_bruto.columns.tolist())


DataFrame com Nomes de Colunas Limpos:
          Mes  Composto R$  Agropecuaria R$  Metal R$  Energia R$  \
0  1998-02-28        38.98            46.62     30.64       18.25   
1  1998-03-31        38.16            45.40     30.70       18.11   
2  1998-04-30        38.10            44.87     30.92       19.57   
3  1998-05-31        38.21            45.28     30.18       19.34   
4  1998-06-30        36.67            43.31     29.57       18.48   

   IC - Composto Var Men (%)  
0                  -0.535851  
1                  -2.103643  
2                  -0.157233  
3                   0.288714  
4                  -4.030359  

Nomes das colunas limpos: ['Mes', 'Composto R$', 'Agropecuaria R$', 'Metal R$', 'Energia R$', 'IC - Composto Var Men (%)']


In [84]:
df_ic_bruto.iloc[83:323,:]

,Mes,Composto R$,Agropecuaria R$,Metal R$,Energia R$,IC - Composto Var Men (%)
83,2005-01-31,101.11,101.36,111.06,90.40,-0.315489
84,2005-02-28,98.34,98.74,109.01,86.39,-2.739591
85,2005-03-31,106.82,106.39,117.55,97.81,8.623144
86,2005-04-30,99.86,99.59,106.79,93.86,-6.515634
87,2005-05-31,93.24,94.57,96.22,84.99,-6.629281
...,...,...,...,...,...,...
318,2024-08-31,404.63,435.97,491.53,194.73,-2.305761
319,2024-09-30,411.26,448.98,502.90,186.18,1.638534
320,2024-10-31,430.22,465.39,535.46,197.67,4.610222
321,2024-11-30,447.00,490.78,530.32,202.69,3.900330


In [87]:
#Criando um novo Dataframe com o mesmo período de tempo do DataFrame df_proxy_custo_ipa (janeiro/2005 - dezembro/2024):

df_ic = df_ic_bruto.iloc[83:323,:].copy()

# Reset do índice do DataFrame::
df_ic.reset_index(drop=True, inplace=True)

df_ic

,Mes,Composto R$,Agropecuaria R$,Metal R$,Energia R$,IC - Composto Var Men (%)
0,2005-01-31,101.11,101.36,111.06,90.40,-0.315489
1,2005-02-28,98.34,98.74,109.01,86.39,-2.739591
2,2005-03-31,106.82,106.39,117.55,97.81,8.623144
3,2005-04-30,99.86,99.59,106.79,93.86,-6.515634
4,2005-05-31,93.24,94.57,96.22,84.99,-6.629281
...,...,...,...,...,...,...
235,2024-08-31,404.63,435.97,491.53,194.73,-2.305761
236,2024-09-30,411.26,448.98,502.90,186.18,1.638534
237,2024-10-31,430.22,465.39,535.46,197.67,4.610222
238,2024-11-30,447.00,490.78,530.32,202.69,3.900330


In [88]:
# Concatenando a coluna "IC - Composto Var Men (%)" do df_ic ao dataframe df_base

df_base = df_base.merge(
    df_ic[["Mes", "IC - Composto Var Men (%)"]],
    on="Mes",
    how="left"
)

df_base

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%,Ind Transf Sem Ajuste Sazonal Var Men (%),ICI,Nivel de estoques,NUCI,IBC-BR Variacao Mensal (%),IC - Composto Var Men (%)
0,2005-01-31,0.250514,-4.469845,106.8,94.8,84.3,-3.756043,-0.315489
1,2005-02-28,0.409522,-5.588296,106.6,92.8,83.9,0.463679,-2.739591
2,2005-03-31,0.754040,15.371508,103.5,91.6,83.4,10.980769,8.623144
3,2005-04-30,0.790196,-1.486933,102.1,90.7,82.2,-2.659851,-6.515634
4,2005-05-31,-0.343708,4.662271,99.8,90.1,81.5,-1.326213,-6.629281
...,...,...,...,...,...,...,...,...
235,2024-08-31,0.607579,1.384228,101.5,96.2,83.2,-1.342374,-2.305761
236,2024-09-30,0.775323,-3.341286,100.4,96.0,83.4,-2.664307,1.638534
237,2024-10-31,0.638631,4.040392,99.9,94.5,82.5,2.037204,4.610222
238,2024-11-30,0.857047,-8.813216,99.1,95.5,81.7,-3.414003,3.900330


### Consumo Aparente da Indústria de Transformação

In [89]:
# Carregamento de Dados a partir de outro arquivo CSV:

df_cons_bruto = pd.read_csv('base_dados_cons_apar.csv')
df_cons_bruto

,Mês,Consumo Aparente - Número índice,Consumo Aparente Dessazonal - Número Índice,Consumo Aparente - Var Mensal,Consumo Aparente Dessazonal - Var Mensal %
0,1998-02-28,62.981360,74.325071,-4.141592,1.032409
1,1998-03-31,75.945699,74.788463,20.584406,0.623467
2,1998-04-30,72.099605,74.201915,-5.064268,-0.784277
3,1998-05-31,76.994793,75.460328,6.789480,1.695932
4,1998-06-30,76.415379,74.498122,-0.752537,-1.275115
...,...,...,...,...,...
320,2024-10-31,104.778050,95.481160,4.818552,2.727584
321,2024-11-30,91.236110,91.986390,-12.924405,-3.660167
322,2024-12-31,80.135030,91.137610,-12.167419,-0.922723
323,2025-01-31,85.654010,93.674320,6.887100,2.783384


In [ ]:
# Remoção de acentos e cedilha dos nomes das colunas:

# Aplicação da função para renomear as colunas para letras sem acento e cedilha
# List comprehension para aplicar a função a cada nome de coluna

df_cons_bruto.columns = [remover_acentos_e_cedilha(col) for col in df_cons_bruto.columns]

print("\nDataFrame com Nomes de Colunas Limpos:")
print(df_cons_bruto.head())
print("\nNomes das colunas limpos:", df_cons_bruto.columns.tolist())


DataFrame com Nomes de Colunas Limpos:
          Mes  Consumo Aparente - Numero indice  \
0  1998-02-28                         62.981360   
1  1998-03-31                         75.945699   
2  1998-04-30                         72.099605   
3  1998-05-31                         76.994793   
4  1998-06-30                         76.415379   

   Consumo Aparente Dessazonal - Numero Indice  Consumo Aparente - Var Mensal  \
0                                    74.325071                      -4.141592   
1                                    74.788463                      20.584406   
2                                    74.201915                      -5.064268   
3                                    75.460328                       6.789480   
4                                    74.498122                      -0.752537   

   Consumo Aparente Dessazonal - Var Mensal %  
0                                    1.032409  
1                                    0.623467  
2                     

In [91]:
df_cons_bruto.iloc[83:323,:]

,Mes,Consumo Aparente - Numero indice,Consumo Aparente Dessazonal - Numero Indice,Consumo Aparente - Var Mensal,Consumo Aparente Dessazonal - Var Mensal %
83,2005-01-31,70.196824,77.823913,-3.616918,-0.839855
84,2005-02-28,65.176955,75.897959,-7.151135,-2.474758
85,2005-03-31,75.841796,76.136751,16.362902,0.314622
86,2005-04-30,74.609993,75.936068,-1.624174,-0.263582
87,2005-05-31,78.388652,76.754420,5.064549,1.077686
...,...,...,...,...,...
318,2024-08-31,103.840780,92.307370,1.948814,-0.091274
319,2024-09-30,99.961360,92.945980,-3.735931,0.691830
320,2024-10-31,104.778050,95.481160,4.818552,2.727584
321,2024-11-30,91.236110,91.986390,-12.924405,-3.660167


In [92]:
#Criando um novo Dataframe com o mesmo período de tempo do DataFrame df_proxy_custo_ipa (janeiro/2005 - dezembro/2024):

df_cons_apar = df_cons_bruto.iloc[83:323,:].copy()

# Reset do índice do DataFrame::
df_cons_apar.reset_index(drop=True, inplace=True)

df_cons_apar

,Mes,Consumo Aparente - Numero indice,Consumo Aparente Dessazonal - Numero Indice,Consumo Aparente - Var Mensal,Consumo Aparente Dessazonal - Var Mensal %
0,2005-01-31,70.196824,77.823913,-3.616918,-0.839855
1,2005-02-28,65.176955,75.897959,-7.151135,-2.474758
2,2005-03-31,75.841796,76.136751,16.362902,0.314622
3,2005-04-30,74.609993,75.936068,-1.624174,-0.263582
4,2005-05-31,78.388652,76.754420,5.064549,1.077686
...,...,...,...,...,...
235,2024-08-31,103.840780,92.307370,1.948814,-0.091274
236,2024-09-30,99.961360,92.945980,-3.735931,0.691830
237,2024-10-31,104.778050,95.481160,4.818552,2.727584
238,2024-11-30,91.236110,91.986390,-12.924405,-3.660167


In [93]:
# Concatenando a coluna "Consumo Aparente - Var Mensal" do df_cons_apar ao dataframe df_base

df_base = df_base.merge(
    df_cons_apar[["Mes", "Consumo Aparente - Var Mensal"]],
    on="Mes",
    how="left"
)

df_base

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%,Ind Transf Sem Ajuste Sazonal Var Men (%),ICI,Nivel de estoques,NUCI,IBC-BR Variacao Mensal (%),IC - Composto Var Men (%),Consumo Aparente - Var Mensal
0,2005-01-31,0.250514,-4.469845,106.8,94.8,84.3,-3.756043,-0.315489,-3.616918
1,2005-02-28,0.409522,-5.588296,106.6,92.8,83.9,0.463679,-2.739591,-7.151135
2,2005-03-31,0.754040,15.371508,103.5,91.6,83.4,10.980769,8.623144,16.362902
3,2005-04-30,0.790196,-1.486933,102.1,90.7,82.2,-2.659851,-6.515634,-1.624174
4,2005-05-31,-0.343708,4.662271,99.8,90.1,81.5,-1.326213,-6.629281,5.064549
...,...,...,...,...,...,...,...,...,...
235,2024-08-31,0.607579,1.384228,101.5,96.2,83.2,-1.342374,-2.305761,1.948814
236,2024-09-30,0.775323,-3.341286,100.4,96.0,83.4,-2.664307,1.638534,-3.735931
237,2024-10-31,0.638631,4.040392,99.9,94.5,82.5,2.037204,4.610222,4.818552
238,2024-11-30,0.857047,-8.813216,99.1,95.5,81.7,-3.414003,3.900330,-12.924405


### Consumo de Energia Elétrica - Indústria

In [94]:
# Carregamento de Dados a partir de outro arquivo CSV:

df_cons_EE_bruto = pd.read_csv('base_dados_cons_EE.csv')
df_cons_EE_bruto

,Mês,Consumo Energia Elétrica Indústria (GWh),Consumo E. Elétrica Ind Var Mensal %
0,1976-02-29,2818.2,0.049702
1,1976-03-31,2951.5,4.729969
2,1976-04-30,2969.0,0.592919
3,1976-05-31,3075.6,3.590434
4,1976-06-30,3110.5,1.134738
...,...,...,...
584,2024-10-31,16880.0,-0.212816
585,2024-11-30,16842.0,-0.225118
586,2024-12-31,16261.0,-3.449709
587,2025-01-31,15985.0,-1.697313


In [95]:
# Remoção de acentos e cedilha dos nomes das colunas:

# Aplicação da função para renomear as colunas para letras sem acento e cedilha
# List comprehension para aplicar a função a cada nome de coluna

df_cons_EE_bruto.columns = [remover_acentos_e_cedilha(col) for col in df_cons_EE_bruto.columns]

print("\nDataFrame com Nomes de Colunas Limpos:")
print(df_cons_EE_bruto.head())
print("\nNomes das colunas limpos:", df_cons_EE_bruto.columns.tolist())


DataFrame com Nomes de Colunas Limpos:
          Mes  Consumo Energia Eletrica Industria (GWh)  \
0  1976-02-29                                    2818.2   
1  1976-03-31                                    2951.5   
2  1976-04-30                                    2969.0   
3  1976-05-31                                    3075.6   
4  1976-06-30                                    3110.5   

   Consumo E. Eletrica Ind Var Mensal %  
0                              0.049702  
1                              4.729969  
2                              0.592919  
3                              3.590434  
4                              1.134738  

Nomes das colunas limpos: ['Mes', 'Consumo Energia Eletrica Industria (GWh)', 'Consumo E. Eletrica Ind Var Mensal %']


In [96]:
df_cons_EE_bruto

,Mes,Consumo Energia Eletrica Industria (GWh),Consumo E. Eletrica Ind Var Mensal %
0,1976-02-29,2818.2,0.049702
1,1976-03-31,2951.5,4.729969
2,1976-04-30,2969.0,0.592919
3,1976-05-31,3075.6,3.590434
4,1976-06-30,3110.5,1.134738
...,...,...,...
584,2024-10-31,16880.0,-0.212816
585,2024-11-30,16842.0,-0.225118
586,2024-12-31,16261.0,-3.449709
587,2025-01-31,15985.0,-1.697313


In [99]:
df_cons_EE_bruto.iloc[347:587,:]

,Mes,Consumo Energia Eletrica Industria (GWh),Consumo E. Eletrica Ind Var Mensal %
347,2005-01-31,12651.0,-5.533154
348,2005-02-28,12747.0,0.758833
349,2005-03-31,13065.0,2.494705
350,2005-04-30,13374.0,2.365098
351,2005-05-31,13292.0,-0.613130
...,...,...,...
582,2024-08-31,17291.0,3.082151
583,2024-09-30,16916.0,-2.168758
584,2024-10-31,16880.0,-0.212816
585,2024-11-30,16842.0,-0.225118


In [100]:
#Criando um novo Dataframe com o mesmo período de tempo do DataFrame df_proxy_custo_ipa (janeiro/2005 - dezembro/2024):

df_cons_EE = df_cons_EE_bruto.iloc[347:587,:].copy()

# Reset do índice do DataFrame::
df_cons_EE.reset_index(drop=True, inplace=True)

df_cons_EE

,Mes,Consumo Energia Eletrica Industria (GWh),Consumo E. Eletrica Ind Var Mensal %
0,2005-01-31,12651.0,-5.533154
1,2005-02-28,12747.0,0.758833
2,2005-03-31,13065.0,2.494705
3,2005-04-30,13374.0,2.365098
4,2005-05-31,13292.0,-0.613130
...,...,...,...
235,2024-08-31,17291.0,3.082151
236,2024-09-30,16916.0,-2.168758
237,2024-10-31,16880.0,-0.212816
238,2024-11-30,16842.0,-0.225118


In [101]:
# Concatenando a coluna "Consumo E. Eletrica Ind Var Mensal %" do df_cons_EE ao dataframe df_base

df_base = df_base.merge(
    df_cons_EE[["Mes", "Consumo E. Eletrica Ind Var Mensal %"]],
    on="Mes",
    how="left"
)

df_base

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%,Ind Transf Sem Ajuste Sazonal Var Men (%),ICI,Nivel de estoques,NUCI,IBC-BR Variacao Mensal (%),IC - Composto Var Men (%),Consumo Aparente - Var Mensal,Consumo E. Eletrica Ind Var Mensal %
0,2005-01-31,0.250514,-4.469845,106.8,94.8,84.3,-3.756043,-0.315489,-3.616918,-5.533154
1,2005-02-28,0.409522,-5.588296,106.6,92.8,83.9,0.463679,-2.739591,-7.151135,0.758833
2,2005-03-31,0.754040,15.371508,103.5,91.6,83.4,10.980769,8.623144,16.362902,2.494705
3,2005-04-30,0.790196,-1.486933,102.1,90.7,82.2,-2.659851,-6.515634,-1.624174,2.365098
4,2005-05-31,-0.343708,4.662271,99.8,90.1,81.5,-1.326213,-6.629281,5.064549,-0.613130
...,...,...,...,...,...,...,...,...,...,...
235,2024-08-31,0.607579,1.384228,101.5,96.2,83.2,-1.342374,-2.305761,1.948814,3.082151
236,2024-09-30,0.775323,-3.341286,100.4,96.0,83.4,-2.664307,1.638534,-3.735931,-2.168758
237,2024-10-31,0.638631,4.040392,99.9,94.5,82.5,2.037204,4.610222,4.818552,-0.212816
238,2024-11-30,0.857047,-8.813216,99.1,95.5,81.7,-3.414003,3.900330,-12.924405,-0.225118


Também será incluida neste dataframe a inflação de bens industrias (IPCA de Bens Industriais), que é a variável target desta previsão.

### IPCA de Bens Industriais

In [ ]:
# Carregamento de Dados a partir de um arquivo Excel:

df_ipca_desag = pd.read_excel('IPCA_VAR Mensal_BC/ipca_desag_var_mensal_maio25.xlsx')
df_ipca_desag.head()

,Mes,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
0,1999-08-31,0.56,1.55,0.31,0.08,0.46,0.10,0.24,0.37,0.17
1,1999-09-30,0.31,0.55,0.36,0.43,0.45,-0.10,0.23,0.37,0.42
2,1999-10-31,1.19,0.75,1.34,0.87,4.39,0.01,1.34,1.77,2.30
3,1999-11-30,0.95,1.86,1.06,0.65,0.68,0.09,0.65,0.52,1.66
4,1999-12-31,0.60,0.67,1.04,0.72,-0.17,0.28,0.58,0.29,1.55


In [103]:
df_ipca_desag

,Mes,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
0,1999-08-31,0.56,1.55,0.31,0.08,0.46,0.10,0.24,0.37,0.17
1,1999-09-30,0.31,0.55,0.36,0.43,0.45,-0.10,0.23,0.37,0.42
2,1999-10-31,1.19,0.75,1.34,0.87,4.39,0.01,1.34,1.77,2.30
3,1999-11-30,0.95,1.86,1.06,0.65,0.68,0.09,0.65,0.52,1.66
4,1999-12-31,0.60,0.67,1.04,0.72,-0.17,0.28,0.58,0.29,1.55
...,...,...,...,...,...,...,...,...,...,...
305,2025-01-31,0.16,-1.52,0.99,0.11,0.36,0.78,0.75,0.47,1.07
306,2025-02-28,1.31,3.16,0.73,0.03,0.47,0.82,0.68,0.39,0.79
307,2025-03-31,0.56,0.18,1.08,0.40,0.21,0.62,0.69,0.37,1.31
308,2025-04-30,0.43,0.35,0.80,0.85,0.39,0.20,0.46,0.60,0.83


In [106]:
df_ipca_desag.iloc[65:305]

,Mes,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
65,2005-01-31,0.58,0.51,0.63,0.48,0.93,0.46,0.61,0.61,0.77
66,2005-02-28,0.59,0.16,0.42,0.13,0.31,1.82,0.77,0.39,0.29
67,2005-03-31,0.61,1.29,0.20,0.55,0.18,0.51,0.33,0.35,0.07
68,2005-04-30,0.87,1.14,0.99,0.95,0.69,0.35,0.76,0.93,0.90
69,2005-05-31,0.49,0.25,0.74,1.14,0.31,0.28,0.59,0.75,0.65
...,...,...,...,...,...,...,...,...,...,...
300,2024-08-31,-0.02,-0.12,-0.54,0.28,0.36,0.24,0.02,0.17,-0.73
301,2024-09-30,0.44,1.01,0.52,0.02,0.03,0.15,0.24,0.16,0.56
302,2024-10-31,0.56,0.71,0.85,0.34,0.33,0.35,0.51,0.26,1.22
303,2024-11-30,0.39,-0.87,1.50,-0.15,-0.12,0.83,0.84,0.19,1.81


In [111]:
#Criando um novo Dataframe com o mesmo período de tempo do DataFrame df_proxy_custo_ipa (janeiro/2005 - dezembro/2024):

df_ipca = df_ipca_desag.iloc[65:305].copy()

# Reset do índice do DataFrame::
df_ipca.reset_index(drop=True, inplace=True)

df_ipca

,Mes,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
0,2005-01-31,0.58,0.51,0.63,0.48,0.93,0.46,0.61,0.61,0.77
1,2005-02-28,0.59,0.16,0.42,0.13,0.31,1.82,0.77,0.39,0.29
2,2005-03-31,0.61,1.29,0.20,0.55,0.18,0.51,0.33,0.35,0.07
3,2005-04-30,0.87,1.14,0.99,0.95,0.69,0.35,0.76,0.93,0.90
4,2005-05-31,0.49,0.25,0.74,1.14,0.31,0.28,0.59,0.75,0.65
...,...,...,...,...,...,...,...,...,...,...
235,2024-08-31,-0.02,-0.12,-0.54,0.28,0.36,0.24,0.02,0.17,-0.73
236,2024-09-30,0.44,1.01,0.52,0.02,0.03,0.15,0.24,0.16,0.56
237,2024-10-31,0.56,0.71,0.85,0.34,0.33,0.35,0.51,0.26,1.22
238,2024-11-30,0.39,-0.87,1.50,-0.15,-0.12,0.83,0.84,0.19,1.81


In [112]:
# Renomeando coluna para manter consistência com o padrão de nomenclatura:

df_ipca = df_ipca.rename(columns={'27863 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Industriais - Var. % mensal': 'IPCA Bens Ind Var Mensal %'
})

In [113]:
df_ipca

,Mes,433 - Indice nacional de precos ao consumidor-amplo (IPCA) - Var. % mensal,4449 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Administrados - Var. % mensal,10841 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens nao-duraveis - Var. % mensal,10842 - Indice de Precos ao Consumidor-Amplo (IPCA) - Bens semi-duraveis - Var. % mensal,10843 - Indice de Precos ao Consumidor-Amplo (IPCA) - Duraveis - Var. % mensal,10844 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Servicos - Var. % mensal,11428 - Indice nacional de precos ao consumidor - Amplo (IPCA) - Itens livres - Var. % mensal,IPCA Bens Ind Var Mensal %,27864 - Indice Nacional de Precos ao Consumidor - Amplo (IPCA) - Alimentacao no domicilio - Var. % mensal
0,2005-01-31,0.58,0.51,0.63,0.48,0.93,0.46,0.61,0.61,0.77
1,2005-02-28,0.59,0.16,0.42,0.13,0.31,1.82,0.77,0.39,0.29
2,2005-03-31,0.61,1.29,0.20,0.55,0.18,0.51,0.33,0.35,0.07
3,2005-04-30,0.87,1.14,0.99,0.95,0.69,0.35,0.76,0.93,0.90
4,2005-05-31,0.49,0.25,0.74,1.14,0.31,0.28,0.59,0.75,0.65
...,...,...,...,...,...,...,...,...,...,...
235,2024-08-31,-0.02,-0.12,-0.54,0.28,0.36,0.24,0.02,0.17,-0.73
236,2024-09-30,0.44,1.01,0.52,0.02,0.03,0.15,0.24,0.16,0.56
237,2024-10-31,0.56,0.71,0.85,0.34,0.33,0.35,0.51,0.26,1.22
238,2024-11-30,0.39,-0.87,1.50,-0.15,-0.12,0.83,0.84,0.19,1.81


In [ ]:
# Concatenando a coluna "IPCA Bens Ind Var Mensal %" do df_ipca ao dataframe df_base

# Garantindo que ambas as colunas 'Mes' estejam no formato datetime
df_base['Mes'] = pd.to_datetime(df_base['Mes'])
df_ipca['Mes'] = pd.to_datetime(df_ipca['Mes'])

df_base = df_base.merge(
    df_ipca[["Mes", "IPCA Bens Ind Var Mensal %"]],
    on="Mes",
    how="left"
)

df_base

,Mes,Proxy - Custo Prod Ind Transf Var Mensal%,Ind Transf Sem Ajuste Sazonal Var Men (%),ICI,Nivel de estoques,NUCI,IBC-BR Variacao Mensal (%),IC - Composto Var Men (%),Consumo Aparente - Var Mensal,Consumo E. Eletrica Ind Var Mensal %,IPCA Bens Ind Var Mensal %
0,2005-01-31,0.250514,-4.469845,106.8,94.8,84.3,-3.756043,-0.315489,-3.616918,-5.533154,0.61
1,2005-02-28,0.409522,-5.588296,106.6,92.8,83.9,0.463679,-2.739591,-7.151135,0.758833,0.39
2,2005-03-31,0.754040,15.371508,103.5,91.6,83.4,10.980769,8.623144,16.362902,2.494705,0.35
3,2005-04-30,0.790196,-1.486933,102.1,90.7,82.2,-2.659851,-6.515634,-1.624174,2.365098,0.93
4,2005-05-31,-0.343708,4.662271,99.8,90.1,81.5,-1.326213,-6.629281,5.064549,-0.613130,0.75
...,...,...,...,...,...,...,...,...,...,...,...
235,2024-08-31,0.607579,1.384228,101.5,96.2,83.2,-1.342374,-2.305761,1.948814,3.082151,0.17
236,2024-09-30,0.775323,-3.341286,100.4,96.0,83.4,-2.664307,1.638534,-3.735931,-2.168758,0.16
237,2024-10-31,0.638631,4.040392,99.9,94.5,82.5,2.037204,4.610222,4.818552,-0.212816,0.26
238,2024-11-30,0.857047,-8.813216,99.1,95.5,81.7,-3.414003,3.900330,-12.924405,-0.225118,0.19


### Criação de Arquivo de Dados em Formato .csv

In [117]:
df_base.to_csv("Dados/Base Completa/base_dados_2005.csv", index=False)